In [ ]:
from langchain.vectorstores import FAISS  # This import should work now
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.document_loaders import WikipediaLoader
from langchain_huggingface import HuggingFacePipeline
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

# Step 1: Load Wikipedia page
def load_wiki(query):
    print(f"🔍 Searching Wikipedia for: {query}")
    loader = WikipediaLoader(query=query, lang="en")
    docs = loader.load()
    print(f"📄 Loaded {len(docs)} Wikipedia document(s).")
    return docs

# Step 2: Split text into chunks
def split_docs(docs):
    print("✂️ Splitting documents into smaller chunks...")
    splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = splitter.split_documents(docs)
    print(f"✅ Created {len(chunks)} text chunks.")
    return chunks

# Step 3: Embed chunks using HuggingFace embeddings
def embed_documents(documents):
    print("🧠 Embedding documents using HuggingFace embeddings...")
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_store = FAISS.from_documents(documents, embeddings)
    print("📦 FAISS vector store created.")
    return vector_store

# Step 4: Load Falcon or Mistral open-source LLM
def load_llm():
    try:
        print("🧠 Loading Falcon-7B-Instruct...")
        model_id = "tiiuae/falcon-7b-instruct"  # or try "mistralai/Mistral-7B-Instruct-v0.1"
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", trust_remote_code=True, torch_dtype=torch.float16)
        pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512, temperature=0.7)
        return HuggingFacePipeline(pipeline=pipe)
    except Exception as e:
        print(f"❌ Error loading model: {e}")
        raise

# Step 5: Create RAG Chatbot
def create_chatbot(query):
    docs = load_wiki(query)
    splitted_docs = split_docs(docs)
    vector_store = embed_documents(splitted_docs)
    llm = load_llm()

    retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        return_source_documents=True
    )
    print("🤖 Chatbot created successfully!")
    return qa_chain

# Step 6: Interactive Chat
def start_chat():
    topic = input("📚 Enter the topic you want to research on Wikipedia: ")
    try:
        chatbot = create_chatbot(topic)
    except Exception as e:
        print(f"❌ Failed to create chatbot: {e}")
        return

    print("\n💬 Ask your questions (type 'exit' to quit):")
    while True:
        question = input("You: ")
        if question.lower() == "exit":
            print("👋 Goodbye!")
            break
        try:
            response = chatbot.invoke({"query": question})  # ✅ invoke instead of deprecated run
            print(f"\n🤖 Bot: {response['result']}\n")

            # Optional: Show sources
            for i, doc in enumerate(response["source_documents"]):
                print(f"📄 Source {i+1}: {doc.metadata.get('title', 'Wikipedia')}")
        except Exception as e:
            print(f"⚠️ Error: {e}")

# Main
if __name__ == "__main__":
    start_chat()

📚 Enter the topic you want to research on Wikipedia:  Dhoni


🔍 Searching Wikipedia for: Dhoni


C:\Users\nares\AppData\Local\Programs\Python\Python312\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file C:\Users\nares\AppData\Local\Programs\Python\Python312\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
Created a chunk of size 2174, which is longer than the specified 500
Created a chunk of size 773, which is longer than the specified 500
Created a chunk of size 1431, which is longer than the specified 500
Created a chunk of size 1154, which is longer than the specified 500
C

📄 Loaded 24 Wikipedia document(s).
✂️ Splitting documents into smaller chunks...
✅ Created 103 text chunks.
🧠 Embedding documents using HuggingFace embeddings...


C:\Users\nares\AppData\Local\Temp\ipykernel_29536\1707039991.py:29: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk and cpu.
Device set to use cpu


🤖 Chatbot created successfully!

💬 Ask your questions (type 'exit' to quit):


You:  Who is Dhoni ?


C:\Users\nares\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\generation\configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
